In [1]:
import pandas as pd
import sqlite3
import numpy as np

# Connect to the SQLite database
conn = sqlite3.connect('statcast.db')

# Define the query to fetch necessary data from the database for Aaron Judge in 2018
query = '''
SELECT 
  strftime('%Y', game_date) AS year, 
  batter_name,
  inning,
  des,
  post_home_score,
  post_away_score,
  on_1b,
  on_2b,
  on_3b,
  outs_when_up
FROM statcast_data
WHERE inning BETWEEN 1 AND 9 AND batter_name LIKE 'Aaron Judge' AND strftime('%Y', game_date) LIKE '2018'
'''

# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Define the function to calculate the new delta_run_exp based on description
def update_run_exp(description):
    description = description.lower()
    points = {
        'walks': 1,
        'single': 2,
        'to 1st.': 2,
        'to 2nd.': 4,
        'doubles': 4,
        'to 3rd.': 6,
        'triples': 6,
        'homers': 10,
        'scores': 7,
        'strikes out': -2,
        'lines out': -2,
        'pops out': -2,
        'called out on strikes': -2,
        'flies out': -2,
        'grounds out': -2,
        'out at 1st.': -2,
        'out at 2nd.': -4,
        'out at 3rd.': -6
    }
    total_points = 0
    for key, value in points.items():
        total_points += description.count(key) * value
    return total_points

# Apply the update_run_exp function to calculate the new 'delta_run_exp'
df['points'] = df['des'].apply(update_run_exp)

# Function to scale points based on outs_when_up
def scale_points(points, outs):
    if points < 0:
        scale_factors = {0: 2, 1: 1.5, 2: 1}
    else:
        scale_factors = {0: 1, 1: 1.5, 2: 2}
    return points * scale_factors[outs]

# Apply the scale_points function
df['points_scaled'] = df.apply(lambda x: scale_points(x['points'], x['outs_when_up']), axis=1)

# Additional required calculations based on the original query structure
df['run_difference'] = (df['post_home_score'] - df['post_away_score']).abs()

# Calculate the frequency of each combination of inning and run_difference
df['frequency'] = df.groupby(['year', 'inning', 'run_difference'])['des'].transform('count')

# Group by inning and run_difference, then aggregate as per the original requirement
result_df = df.groupby(['year', 'inning', 'run_difference']).agg(
    avg_delta_run_exp=pd.NamedAgg(column='points_scaled', aggfunc='mean'),
    frequency=pd.NamedAgg(column='frequency', aggfunc='max')
).reset_index()

# Calculate arcsinh of average delta_run_exp
result_df['avg_delta_ihs'] = np.arcsinh(result_df['avg_delta_run_exp'])

# Save the DataFrame to a CSV file
result_df.to_csv('C:\\Users\\17323\\Desktop\\430_Final_Project\\FINALONE.csv', index=False)

print("Dataframe with run differential and frequency created and saved.")

# Close the connection to the database
conn.close()


DatabaseError: Execution failed on sql '
SELECT 
  strftime('%Y', game_date) AS year, 
  batter_name,
  inning,
  des,
  post_home_score,
  post_away_score,
  on_1b,
  on_2b,
  on_3b,
  outs_when_up
FROM statcast_data
WHERE inning BETWEEN 1 AND 9 AND batter_name LIKE 'Aaron Judge' AND strftime('%Y', game_date) LIKE '2018'
': no such table: statcast_data